### 任务描述
利用训练的模型来预测数据所属的类别。

### 数据说明
本数据集包含12种类的猫的图片。

整个数据将被分为训练集与测试集。在训练数据中，我们提供彩色的图片，如图所示。

训练集：在训练集中，我们将提供高清彩色图片以及图片所属的分类。

测试集：在测试数据集中，我们仅仅提供彩色图片。



最终精度：0.97

## 安装PaddleX

In [3]:
!pip install paddlex
# 每次启动都要重新运行

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 206.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 613.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 363.4 kB/s eta 0:00:0000:0100:01
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593914 sha256=4a307a85498be85d928840cface4d77f09be5fdce14e2661

## 导入相关包（由于版本问题需要去运行下面语句，将paddlelim版本换成2.1.1）

In [4]:
pip install paddleslim==2.1.1


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 461.5 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddleslim
    Found existing installation: paddleslim 2.2.1
    Uninstalling paddleslim-2.2.1:
      Successfully uninstalled paddleslim-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlex 2.1.0 requires paddleslim==2.2.1, but you have paddleslim 2.1.1 which is incompatible.

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import warnings
warnings.filterwarnings('ignore') # 忽略警告信息
()
import os
from paddlex import transforms as T # 用于定义模型训练、验证、预测过程中，输入图像的预处理和数据增强操作
                                    
import paddlex as pdx
import paddle
from paddle.regularizer import L2Decay # L2 权重衰减正则化     

import numpy as np
import pandas as pd
import shutil # 文件文档处理库
import cv2    
import imghdr # 检测图片类型
from PIL import Image
from matplotlib import pyplot as plt

[11-04 15:44:07 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

# 数据分析与处理
## 数据导入（解压数据集：这个平台可以直接书写linux语句）

In [6]:
!unzip -q /home/aistudio/data/data10954/cat_12_train.zip -d data/data10954/
!unzip -q /home/aistudio/data/data10954/cat_12_test.zip -d data/data10954/

In [7]:
## 相关文件夹的删除与建立
!rm -rf data/data10954/ImageNetDataset # 删除文件夹，防止多次运行时出错

for i in range(12):
    cls_path = os.path.join('data/data10954/ImageNetDataset/', '%02d' % int(i)) # 拼接路径
    if not os.path.exists(cls_path):
        os.makedirs(cls_path) # 创建文件夹

!ls data/data10954/ImageNetDataset # 列出文件夹（linux语句）

##生成文件名和类别的一一对应关系，之后将根据类别cls将图片放入目标文件夹：data/data10954/ImageNetDataset/*/*.jpg。
train_df = pd.read_csv('data/data10954/train_list.txt', header=None, sep='\t') # 读取测试集标签
train_df.columns = ['name', 'cls'] # 返回列索引列表
train_df['name'] = train_df['name'].apply(lambda x: str(x).strip().split('/')[-1]) # 切分文件名，舍去cat_12_train/
train_df['cls'] = train_df['cls'].apply(lambda x: '%02d' % int(str(x).strip())) # 使图片标签类别变成2位数字
train_df.head(20) # 观察前五行数据格式

00  01	02  03	04  05	06  07	08  09	10  11


,name,cls
0,8GOkTtqw7E6IHZx4olYnhzvXLCiRsUfM.jpg,00
1,hwQDH3VBabeFXISfjlWEmYicoyr6qK1p.jpg,00
2,RDgZKvM6sp3Tx9dlqiLNEVJjmcfQ0zI4.jpg,00
3,ArBRzHyphTxFS2be9XLaU58m34PudlEf.jpg,00
4,kmW7GTX6uyM2A53NBZxibYRpQnIVatCH.jpg,00
5,G4RDpHPcFZmwK07MlLWdgTAvXrutOkQj.jpg,00
6,cIfTNJzpynxK4FmbsZG0Wuia8A9BD5gk.jpg,00
7,N5XosuxBjUTGfWd8kvFS9q2YDHyK7pMr.jpg,00
8,tvLoIG6cSO3Mbz0iugNZpFDJq54dHjxm.jpg,00
9,i8PW0FObIvcrETzZuRHha3DeKko79mtj.jpg,00


## 图片模式检验&修复

图片模式主要有以下几种：  
1、RGB 为真色彩模式， 可组合为 256 x 256 x256 种， 打印需要更改为 CMYK模式， 需要注意数值溢出的问题。  
2、HSB 模式（本篇没有涉及），建立基于人类感觉颜色的方式，将颜色分为色相（Hue），饱和度（Saturation），明亮度（Brightness），这里不详细展开。  
3、CMYK模式，应用在印刷领域，4个字母意思是青、洋红、黄、黑，因为不能保证纯度，所以需要黑。  
4、位图模式，见1， 颜色由黑和白表示（True， False）。  
5、灰度模式，只有灰度， 所有颜色转化为灰度值，见L，I，F。  
6、双色调模式（未有涉及），节约成本将可使用双色调。  
7、Lab模式（未涉及，ps内置），由3通道组成（亮度，a，b）组成，作为RGB到CMYK的过渡。  
8、多通道模式，删除RGB，CMYK，Lab中某一个通道后，会转变为多通道，多通道用于处理特殊打印，它的每个通道都为256级灰度通道。  
9、索引颜色模式，用在多媒体和网页，通过颜色表查取，没有则就近取，仅支持单通道，（8位/像素）。   
</br>
通过对数据集图片模式进行检验，我们发现其含有 **‘P’,’RGBA’,’RGB’** 三种不同模式的图片。  
P（pallete）模式：调色板模式，把原来单像素占用24(32)个bit的RGB(A)真彩图片中的像素值，重映射到了8bit长，即0~255的数值范围内。而这套映射关系，就是属于这张图的所谓“调色板”(Pallete)。

In [8]:
## 图片格式应当为RGB三通道，其中一张RGBA模式图片展示如下
img = Image.open('data/data10954/cat_12_train/ulFBEZNRQrxn57voHAJ4UG6Mct2sw1Cj.jpg')
print(img.mode)
plt.imshow(img)
plt.show(img)

RGBA


In [9]:
## P、RGBA、L模式的图片转换为RGB模式
for i in range(len(train_df)):
    img_path = os.path.join('data/data10954/cat_12_train', train_df.at[i, 'name']) # i 元素在列中的位置 ，name 列名
    if os.path.exists(img_path) and imghdr.what(img_path): # 检测路径文件是否存在及判断类别
        img = Image.open(img_path) # 打开文件
        if img.mode != 'RGB':
            img = Image.open(img_path)
            print(img_path)
            print(img.mode)
            img = img.convert('RGB') # 转换成rgb形式
            img.save(img_path) # 保存
          
for img_path in os.listdir('data/data10954/cat_12_test'):
    src = os.path.join('data/data10954/cat_12_test',img_path)
    img = Image.open(src)
    if img.mode != 'RGB':
        print(img_path)
        img = img.convert('RGB') 
        img.save(src)

data/data10954/cat_12_train/tO6cKGH8uPEayzmeZJ51Fdr2Tx3fBYSn.jpg
P
data/data10954/cat_12_train/ulFBEZNRQrxn57voHAJ4UG6Mct2sw1Cj.jpg
RGBA
data/data10954/cat_12_train/F3VnNwb2K9tgMWLodrXl1f6PIEjYqhy8.jpg
L
data/data10954/cat_12_train/YfsxcFB9D3LvkdQyiXlqnNZ4STwope2r.jpg
P
data/data10954/cat_12_train/6yYs4rvFLkQJlRxdhNfMOW52EAbgHejC.jpg
RGBA
data/data10954/cat_12_train/5nKsehtjrXCZqbAcSW13gxB8E6z2Luy7.jpg
P
data/data10954/cat_12_train/yGcJHV8Uuft6grFs7QWnK5CTAZvYzdDO.jpg
P
data/data10954/cat_12_train/YGyx4qCdOb7j8tzBuNfoFHLi6gU0SE3T.jpg
RGBA
data/data10954/cat_12_train/3yMZzWekKmuoGOF60ICQxldhBEc9Ra15.jpg
P
Qt29gPjYZwv3B6RJh5yiTWXrVImue1FH.jpg


## 数据可视化

In [10]:
## Data Visualization
## 随机查看同一类猫咪的特征
plt.figure(1)
img_1_1 = Image.open('data/data10954/cat_12_train/spNU7J8uk6BXiAyQErHegYMzjOaFR2qV.jpg')
plt.subplot(2, 2, 1) #图一包含1行2列子图，当前画在第一行第一列图上
plt.imshow(img_1_1)
plt.subplot(2, 2, 2)#当前画在第一行第2列图上
img_1_2 = Image.open('data/data10954/cat_12_train/7QZTYlspK2fqdJUwjC0HDmOFrM5W4PX9.jpg')
plt.imshow(img_1_2)
plt.subplot(2, 2, 3)
img_1_3 = Image.open('data/data10954/cat_12_train/oZin4PuwTet39xWCYhUBfvlzGyISb5DV.jpg')
plt.imshow(img_1_3)
plt.subplot(2, 2, 4)
img_1_4 = Image.open('data/data10954/cat_12_train/qbKjsR05lrFVYfLChtMGD7im36cUgAnE.jpg')
plt.imshow(img_1_4)

In [11]:
## 随机选取不同类别的猫咪进行查看
plt.figure(2)
img_0 = Image.open('data/data10954/cat_12_train/8GOkTtqw7E6IHZx4olYnhzvXLCiRsUfM.jpg')
plt.subplot(2, 6, 1)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/spNU7J8uk6BXiAyQErHegYMzjOaFR2qV.jpg')
plt.subplot(2, 6, 2)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/jbIdxGyNpoql3XQZrfREMiAzh7B46WOa.jpg')
plt.subplot(2, 6, 3)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/cCeBo4EJ9H1hbXsIS5G6Kxdzg27nwqfy.jpg')
plt.subplot(2, 6, 4)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/yxNcRSz4TI7FpwCVJBuea6MmGitZYUkK.jpg')
plt.subplot(2, 6, 5)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/NZw3P0Wfz4JDsSECG8y7HXihl2Oon6rA.jpg')
plt.subplot(2, 6, 6)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/K5wdv0zEnx3cti4OagyPphCVJUIXYuSZ.jpg')
plt.subplot(2, 6, 7)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/BOmo5yiKzMGV8qvleRIdLQC4bZcPxwWD.jpg')
plt.subplot(2, 6, 8)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/COJUByb07wYXqcTMovWFnAgpNZk1SxrI.jpg')
plt.subplot(2, 6, 9)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/dHJn0vb8XoSTM4DPG965fQ1swczARBel.jpg')
plt.subplot(2, 6, 10)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/sv3RcZgEInHWtBoVKr9Q46PMUmA8Jy2h.jpg')
plt.subplot(2, 6, 11)
plt.imshow(img_0)
img_0 = Image.open('data/data10954/cat_12_train/mrgAsyPJdDvwp1EYnUG3Hj92ehMTKNxt.jpg')
plt.subplot(2, 6, 12)
plt.imshow(img_0)


In [12]:
! pip install pyecharts

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/135.6 kB ? eta -:--:--━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/135.6 kB 1.6 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 71.7/135.6 kB 1.2 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 112.6/135.6 kB 1.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 122.9/135.6 kB 359.7 kB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 122.9/135.6 kB 359.7 kB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 122.9/135.6 kB 359.7 kB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━

## 样本平衡问题检验（此图像去找work文件夹中产生的labels.html打开即可看到）
为了检验或解决本项目样本不均匀问题，我们对data/cat_12_train中各类猫的图片数量进行统计并绘制条形图，结果如下图所示。由此可以观察出，本项目不存在样本不均衡问题，可直接进行下一步操作。
![](https://ai-studio-static-online.cdn.bcebos.com/e644e5a8a89e4c06b47073caa94e4f191c64fd9891e34dc7852e6c8ba2faafdb)  

In [15]:
## 统计训练集各类猫的数目，防止样本不平衡问题。
from pyecharts import options as opts
from pyecharts.charts import Bar

with open("data/data10954/train_list.txt", "r") as f:
    labels = f.readlines()
    labels = [int(i.split()[-1]) for i in labels]

counts = pd.Series(labels).value_counts().sort_index().to_list()
values = np.random.rand(12) * 100
names = [str(i) for i in list(range(12))]
data = list(zip(values, counts, names))
source = [list(i) for i in data]
source.insert(0, ["score", "amount", "product"])


c = (
    Bar()
    .add_dataset(
        source=source
    )
    .add_yaxis(
        series_name="",
        y_axis=[],
        encode={"x": "amount", "y": "product"},
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Dataset normal bar example"),
        xaxis_opts=opts.AxisOpts(name="amount"),
        yaxis_opts=opts.AxisOpts(type_="category"),
        visualmap_opts=opts.VisualMapOpts(
            orient="horizontal",
            pos_left="center",
            min_=10,
            max_=100,
            range_text=["High Score", "Low Score"],
            dimension=0,
            range_color=["#D7DA8B", "#E15457"],
        ),
    )
    .render("./work/labels.html")
)


In [16]:
## 从源路径 src_path 移动至目标路径 dst_path。
for i in range(len(train_df)):
    # 源路径
    src_path = os.path.join('data/data10954/cat_12_train',train_df.at[i, 'name']) # i 元素在列中的位置 ，name 列名
    # 目标路径
    dst_path = os.path.join(os.path.join('data/data10954/ImageNetDataset/',train_df.at[i, 'cls']),train_df.at[i, 'name'])
    try:
        shutil.move(src_path, dst_path) # 移动图片到目标路径
    except Exception as e: 
        print(e) # 抛出错误信息

## 数据增强
在图像分类任务中，图像数据的增广是一种常用的正则化方法，常用于数据量不足或者模型参数较多的场景。在本章节中，我们将对除 ImageNet 分类任务标准数据增强外的8种数据增强方式进行简单的介绍和对比，用户也可以将这些增广方法应用到自己的任务中，以获得模型精度的提升。这8种数据增强方式在ImageNet上的精度指标如下所示。
<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/613d6ebfebc04ae181e712ea743023a190b73c66ec334963bd716d8484349a05" />
</div>

ImageNet 分类训练阶段的标准数据增强方式分为以下几个步骤：
1. 图像解码：简写为 ImageDecode
2. 随机裁剪到长宽均为 224 的图像：简写为 RandCrop
3. 水平方向随机翻转：简写为 RandFlip
4. 图像数据的归一化：简写为 Normalize
5. 图像数据的重排，[224, 224, 3] 变为 [3, 224, 224]：简写为 Transpose
6. 多幅图像数据组成 batch 数据，如 batch-size 个 [3, 224, 224] 的图像数据拼组成 [batch-size, 3, 224, 224]：简写为 Batch
下图为三类数据增强方式的效果展示：
![](https://ai-studio-static-online.cdn.bcebos.com/9cd7a6cf710b4ba998b201821226b8c4d7a796e0f4c5419699d69e6efb8ad1fb)
**图像变换类**：图像变换类是在随机裁剪与翻转之间进行的操作，也可以认为是在原图上做的操作。主要方式包括AutoAugment和RandAugment，基于一定的策略，包括锐化、亮度变化、直方图均衡化等，对图像进行处理。这样网络在训练时就已经见过这些情况了，之后在实际预测时，即使遇到了光照变换、旋转这些很棘手的情况，网络也可以从容应对了。  
**图像裁剪类**：图像裁剪类主要是在生成的在通道转换之后，在图像上设置掩码，随机遮挡，从而使得网络去学习一些非显著性的特征。否则网络一直学习很重要的显著性区域，之后在预测有遮挡的图片时，泛化能力会很差。主要方式包括：CutOut、RandErasing、HideAndSeek、GridMask。这里需要注意的是，在通道转换前后去做图像裁剪，其实是没有区别的。因为通道转换这个操作不会修改图像的像素值。  
**图像混叠类**：组完batch之后，图像与图像、标签与标签之间进行混合，形成新的batch数据，然后送进网络进行训练。这也就是图像混叠类数据增广方式，主要的有Mixup与Cutmix两种方式。  
由于本项目数据集中，包含2160张训练集图片，为了增强模型效果，我们采用数据增强，并通过调整方式及相关参数，使模型效果最优（相关设置请见配置文件）。  

**我们以训练集中7QZTYlspK2fqdJUwjC0HDmOFrM5W4PX9.jpg为例，展示数据增强效果：**  
**（注：由于前期使用Pytorch进行模型建立，故将相关代码放于图片下方。）**  

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/52322333017241dab060280e6b25cfb924a5414a7ff9454c844c8992ae669532" />
</div>

**相关代码如下：**  
```python
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm import data

# 数据增强策略
trans = transforms.Compose([
    transforms.RandomCrop((384, 384), pad_if_needed=True),
    transforms.RandomHorizontalFlip(),
    data.AutoAugment(data.auto_augment_policy('originalr')),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    transforms.RandomErasing()
])

# 加载单张图片
image = Image.open("./0yTr3fswKBv4M8Fo2NcUnzibx6ClIm5e.jpg.jpg")
image = image.convert("RGB")

# 标准化的参数
mean = np.array(IMAGENET_DEFAULT_MEAN)
std = np.array(IMAGENET_DEFAULT_STD)

# 进行绘图
plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    # 对图片进行增强
    trans_image = trans(image)
    # 提取增强后的图片，转换为numpy.ndarray格式
    trans_image = trans_image.numpy().transpose([1, 2, 0])
    # 反标准化
    trans_image = std * trans_image + mean
    trans_image = np.clip(trans_image, 0, 1)
    # 展示图片
    plt.imshow(trans_image)
    plt.axis('off')

plt.savefig('./src/0yTr3fswKBv4M8Fo2NcUnzibx6ClIm5e.jpg_2.jpg', dpi=100)
plt.show()
```

# 模型构建

## 模型概述（ResNet）
**ResNet的诞生**
深度网络的退化问题（Degradation problem）：网络深度增加时，网络准确度出现饱和，甚至出现下降。深度网络的退化问题至少说明深度网络不容易训练。但是我们考虑这样一个事实：现在你有一个浅层网络，你想通过向上堆积新层来建立深层网络，一个极端情况是这些增加的层什么也不学习，仅仅复制浅层网络的特征，即这样新层是恒等映射（Identity mapping）。在这种情况下，深层网络应该至少和浅层网络性能一样，也不应该出现退化现象。为了解决这个问题，ResNet的作者何凯明提出了残差学习来解决退化问题。  
对于一个堆积层结构（几层堆积而成）当输入为 时其学习到的特征记为 ，现在我们希望其可以学习到残差   ，这样其实原始的学习特征是   。之所以这样是因为残差学习相比原始特征直接学习更容易。当残差为0时，此时堆积层仅仅做了恒等映射，至少网络性能不会下降，实际上残差不会为0，这也会使得堆积层在输入特征基础上学习到新的特征，从而拥有更好的性能。残差学习的结构如下图所示。这有点类似与电路中的“短路”，所以是一种短路连接（shortcut connection）。  
![](https://ai-studio-static-online.cdn.bcebos.com/0ecd6553881040fb9f63991cecc8ff8866f08a0ec5534529869ca1898ed9125b)  
ResNet网络是参考了VGG19网络，在其基础上进行了修改，并通过短路机制加入了残差单元，如图5所示。变化主要体现在ResNet直接使用stride=2的卷积做下采样，并且用global average pool层替换了全连接层。ResNet的一个重要设计原则是：当feature map大小降低一半时，feature map的数量增加一倍，这保持了网络层的复杂度。从图5中可以看到，ResNet相比普通网络每两层间增加了短路机制，这就形成了残差学习，其中虚线表示feature map数量发生了改变。图5展示的34-layer的ResNet，还可以构建更深的网络如表1所示。从表中可以看到，对于18-layer和34-layer的ResNet，其进行的两层间的残差学习，当网络更深时，其进行的是三层间的残差学习，三层卷积核分别是1x1，3x3和1x1，一个值得注意的是隐含层的feature map数量是比较小的，并且是输出feature map数量的1/4。  
![](https://ai-studio-static-online.cdn.bcebos.com/487e996065914e62a8c410b1163981d19bd4d07932854503a7db243cdeeedc3d)


# PaddleClas训练模型
PaddleClas支持通过修改配置文件(.yaml)的方式，灵活便捷的配置模型训练参数。相关配置文件已放于相应文件夹下。本文重点介绍全局配置(Global)、优化器(Optimizer)相关参数。  
## 全局配置相关参数

| 参数名字 | 参数名字	 | 默认值 | 可选值 |
| :------: | :-------: | :-------: | :-------: |
| checkpoints | 断点模型路径，用于恢复训练 | null | str |
| pretrained_model | 预训练模型路径 | null | str|
| output_dir | output_dir |	"./output/" | str |
| save_interval | save_interval | 1 | int |
| eval_during_train | 是否在训练时进行评估 | True | bool |
| eval_interval	| 每隔多少个epoch进行模型评估 | 1 | int |
| epochs | 训练总epoch数 |  | int |
| print_batch_step | print_batch_step| 10 |	int |
| use_visualdl | 是否是用visualdl可视化训练过程 | False | bool |
| image_shape | 图片大小 | [3，224，224] | list, shape: (3,) |
| save_inference_dir | save_inference_dir | "./inference" | str |
| eval_mode | eval的模式 |	"classification" | "retrieval" |
| to_static | 是否改为静态图模式 | False | True |
| ues_dali | 是否使用dali库进行图像预处理 | False | True |
	  



## 优化器相关参数


| 参数名字 | 具体含义 | 默认值 | 可选值 |
| :--------: | :--------: | :--------: | :---------: |
|  name   | 优化器方法名 | "Momentum" | "Momentum" |
| momentum | momentum值 | 0.9| float |
| lr.name | 学习率下降方式 | "Cosine" | "Linear"、"Piecewise"等其他下降方式 |
| lr.learning_rate | 学习率初始值	| 0.1 | float |
| lr.warmup_epoch | warmup轮数 | 0 | int，如5 |
| regularizer.name | 正则化方法名	| "L2" | ["L1", "L2"] |
| regularizer.coeff	| 正则化系数 | 0.00007 | float|

## 生成ImageNet

## [3] 数据集划分

In [12]:
!paddlex --split_dataset --format ImageNet\
    --dataset_dir data/data10954/ImageNetDataset\
    --val_value 0.085\
    --test_value 0

[10-28 20:25:50 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format ImageNet --dataset_dir data/data10954/ImageNetDataset --val_value 0.085 --test_value 0
[10-28 20:25:50 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specificall

## [4] 定义数据增强、装载数据集

在对数据集进行数据增强之前，我们首先需要根据本数据集计算相关参数：  
**T.Normalize()：**  
初始参数：  
**T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])**  
为在ImageNet上几百万张图片上的均值和方差，我们需计算本数据集上的均值和方差，并用此数据进行标准化。  
```python
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from dataset.CatDataset import CatDataset

trans = transforms.Compose([
    transforms.Resize((410, 410)),
    transforms.ToTensor()
])

train_dataset = CatDataset("E:/project/data10954", "train_list.txt", trans)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)


def get_mean_std(loader):
    # Var[x] = E[X**2]-E[X]**2
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    print(num_batches)
    print(channels_sum)
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std


mean, std = get_mean_std(train_loader)

print(mean)
print(std)

```
相关代码如上，经计算，本数据集的相关参数：  
**mean = [0.4848, 0.4435, 0.4023],   
std = [0.2744, 0.2688, 0.2757]**

In [13]:
# 训练集增强
train_transforms = T.Compose([
    T.MixupImage(
        alpha=1.5,
        beta=1.5,
        mixup_epoch=int(300 * 25. / 27)),
    T.Resize(
        target_size=438,
        interp='CUBIC'),
    # 以图像中心点扩散裁剪长宽为目标尺寸的正方形
    T.RandomCrop(360),
    # 以一定的概率对图像进行随机水平翻转
    T.RandomHorizontalFlip(0.5),
    # 以一定的概率对图像进行随机像素内容变换，可包括亮度、对比度、饱和度、色相角度、通道顺序的调整，模型训练时的数据增强操作
    T.RandomDistort(
        brightness_range=0.25,
        brightness_prob=0.5,
        contrast_range=0.25,
        contrast_prob=0.5,
        saturation_range=0.25,
        saturation_prob=0.5,
        hue_range=18.0,
        hue_prob=0.5),
    # 以一定的概率对图像进行高斯模糊
    T.RandomBlur(0.1),
    # 对图像进行标准化
    T.Normalize([0.4848, 0.4435, 0.4023], [0.2744, 0.2688, 0.2757])
])
# 验证集增强
eval_transforms = T.Compose([
    T.Resize(
        target_size=410,
        interp='AREA'),
    T.CenterCrop(360),
    T.Normalize([0.4848, 0.4435, 0.4023], [0.2744, 0.2688, 0.2757])
])

## 装载数据集

In [14]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='data/data10954/ImageNetDataset',
    file_list='data/data10954/ImageNetDataset/train_list.txt',
    label_list='data/data10954/ImageNetDataset/labels.txt',
    transforms=train_transforms,
    shuffle=True) # 是否需要对数据集中样本打乱顺序

eval_dataset = pdx.datasets.ImageNet(
    data_dir='data/data10954/ImageNetDataset',
    file_list='data/data10954/ImageNetDataset/val_list.txt',
    label_list='data/data10954/ImageNetDataset/labels.txt',
    transforms=eval_transforms)

2022-10-28 20:26:11 [INFO]	Starting to read file list from dataset...
2022-10-28 20:26:11 [INFO]	1980 samples in file data/data10954/ImageNetDataset/train_list.txt
2022-10-28 20:26:11 [INFO]	Starting to read file list from dataset...
2022-10-28 20:26:11 [INFO]	180 samples in file data/data10954/ImageNetDataset/val_list.txt


## 配置 ResNet 模型并训练

In [15]:
print(train_dataset.labels)

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']


In [ ]:
#初始化模型
model = pdx.cls.ResNet101_vd_ssld(
    num_classes=len(train_dataset.labels)
)

W1027 15:12:05.698530    98 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W1027 15:12:05.701606    98 device_context.cc:422] device: 0, cuDNN Version: 8.2.


In [ ]:
model.train(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    num_epochs=420, #训练轮数
    train_batch_size=80, #一个step所用到的样本量
    warmup_steps=(len(train_dataset.file_list) // 80) * 6, #学习率从0经过steps轮迭代增长到设定的学习率
    learning_rate=0.025, # 学习率
    lr_decay_epochs=[40, 65, 115, 160, 205], #表示学习率在第几个epoch时衰减一次
    lr_decay_gamma=0.1, # 学习率衰减率

    save_interval_epochs=2, # 每几轮保存一次
    log_interval_steps=(len(train_dataset.file_list) // 80) * 7, # 训练日志输出间隔

    pretrain_weights='IMAGENET',
    #pretrain_weights (str or None): 若指定为'.pdparams'文件时，则从文件加载模型权重；
    #若为字符串'IMAGENET'，则自动下载在ImageNet图片数据上预训练的模型权重；
    #若为None，则不使用预训练模型。默认为'IMAGENET'
    save_dir='output/ResNet101_vd_ssld',
    use_vdl=False)

2022-10-27 15:12:07 [INFO]	Loading pretrained model from output/ResNet101_vd_ssld/pretrain/ResNet101_vd_ssld_pretrained.pdparams
2022-10-27 15:12:09 [WARNING]	[SKIP] Shape of pretrained params fc.weight doesn't match.(Pretrained: (2048, 1000), Actual: [2048, 12])
2022-10-27 15:12:09 [WARNING]	[SKIP] Shape of pretrained params fc.bias doesn't match.(Pretrained: (1000,), Actual: [12])
2022-10-27 15:12:09 [INFO]	There are 530/532 variables loaded into ResNet101_vd_ssld.
2022-10-27 15:12:33 [INFO]	[TRAIN] Epoch 1 finished, loss=2.428278, acc1=0.14687501, acc5=0.5494792 .
2022-10-27 15:12:56 [INFO]	[TRAIN] Epoch 2 finished, loss=1.4307181, acc1=0.6453125, acc5=0.95416665 .
2022-10-27 15:12:56 [INFO]	Start to evaluate(total_samples=180, total_steps=3)...
2022-10-27 15:12:58 [INFO]	[EVAL] Finished, Epoch=2, acc1=0.900000, acc5=0.995833 .
2022-10-27 15:12:59 [INFO]	Model saved in output/ResNet101_vd_ssld/best_model.
2022-10-27 15:12:59 [INFO]	Current evaluated best model on eval_dataset is epo

## 训练技巧与参数选择
### 调优策略
在训练网络的过程中，通常会打印每一个epoch的训练集准确率和验证集准确率，二者刻画了该模型在两个数据集上的表现。通常来说，训练集的准确率比验证集准确率微高或者二者相当是比较不错的状态。如果发现训练集的准确率比验证集高很多，说明在这个任务上已经过拟合，需要在训练过程中加入更多的正则，如增大l2_decay的值，加入更多的数据增广策略，加入label_smoothing策略等；如果发现训练集的准确率比验证集低一些，说明在这个任务上可能欠拟合，需要在训练过程中减弱正则效果，如减小l2_decay的值，减少数据增广方式，增大图片crop区域面积，减弱图片拉伸变换，去除label_smoothing等。

### 优化器&学习率选择
**学习率下降策略：**
在整个训练过程中，我们不能使用同样的学习率来更新权重，否则无法到达最优点，所以需要在训练过程中调整学习率的大小。在训练初始阶段，由于权重处于随机初始化的状态，损失函数相对容易进行梯度下降，所以可以设置一个较大的学习率。在训练后期，由于权重参数已经接近最优值，较大的学习率无法进一步寻找最优值，所以需要设置一个较小的学习率。  
Cosine_decay和piecewise_decay的学习率变化曲线如下图所示，容易观察到，在整个训练过程中，cosine_decay都保持着较大的学习率，所以其收敛较为缓慢，但是最终的收敛效果较peicewise_decay更好一些。
![](https://ai-studio-static-online.cdn.bcebos.com/84eec8a5a72a4a10a453a3228bea612ab0bb647fb80e417bbf7c77c155ac2c1c)



**warmup策略：**
让学习率先进行预热，在训练初期，本文不直接使用最大的学习率，而是用一个逐渐增大的学习率去训练网络，当学习率增大到最高点时，再使用学习率下降策略中提到的学习率下降方式衰减学习率的值。  

```
Optimizer:
  name: AdamW
  beta1: 0.9
  beta2: 0.999
  epsilon: 1e-8
  weight_decay: 0.05
  no_weight_decay_name: absolute_pos_embed relative_position_bias_table .bias norm 
  one_dim_param_no_weight_decay: True
  lr:
    name: Cosine
    learning_rate: 3e-6
    eta_min: 1e-6
    warmup_epoch: 20
    warmup_start_lr: 1e-6
```

### batch_size
batch_size决定了一次将多少数据送入神经网络参与训练，当batch_size的值与学习率的值呈线性关系时，收敛精度几乎不受影响。因本文采用飞桨—至尊版GPU环境，所以在条件的允许下，尽量增大batch_size值的大小  
（从64开始尝试，若训练过程中出现内存溢出错误，则减小batch_size值的大小。）

## 模型预测，另存为提交文件

In [ ]:
model = pdx.load_model('output/ResNet101_vd_ssld/epoch_40') # 加载模型
model.get_model_info() # 显示信息

## 生成 work/result.csv 
通过csv文件去查看分类结果，分类结果并不是100%正确，在有经过数据增强和神经网络的学习前，正确率大约在20%，经过数据增强和残差神经网络学习以后可以到达90%左右，但是这其中也不排除训练次数太多导致的过拟合，最终使正确率降低了一些。

In [ ]:
import glob

test_list = glob.glob('data/data10954/cat_12_test/*.jpg')
test_df = pd.DataFrame() # 创建表结构

for i in range(len(test_list)):
    img = Image.open(test_list[i]).convert('RGB')
    img = np.asarray(img, dtype='float32') # 转换数据类型

    result = model.predict(img[:, :, [2, 1, 0]]) # 预测结果
    test_df.at[i, 'name'] = str(test_list[i]).split('/')[-1] # 文件名
    test_df.at[i, 'cls'] = int(result[0]['category_id']) # 类别

test_df[['name']] = test_df[['name']].astype(str)
test_df[['cls']] = test_df[['cls']].astype(int)
test_df.to_csv('work/result.csv', index=False, header=False) # 生成csv文件

test_df.head()